In [1]:
from __future__ import print_function
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import seaborn as sns
import gc

import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score
from scipy.stats import norm, rankdata

import keras
from keras import regularizers
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add,PReLU, LSTM
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Reshape, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import tensorflow as tf
import horovod.keras as hvd

Using TensorFlow backend.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [2]:
# reduce memory
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [3]:
# Horovod: initialize Horovod.
hvd.init()

# Horovod: pin GPU to be used to process local rank (one GPU per process)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = str(hvd.local_rank())
K.set_session(tf.Session(config=config))

In [4]:
train = reduce_mem_usage(pd.read_csv('../input/train.csv'))
test = reduce_mem_usage(pd.read_csv('../input/test.csv'))

Memory usage after optimization is: 78.01 MB
Decreased by 74.7%
Memory usage after optimization is: 77.82 MB
Decreased by 74.6%


In [5]:
features = [f for f in train if f not in ['ID_code','target']]

In [6]:
df_original = pd.concat([train, test],axis=0,sort=False)
df = df_original[features]
target = df_original['target'].values
id = df_original['ID_code']

In [7]:
for feature in features:
    df['mean_'+feature] = (train[feature].mean()-train[feature])
    df['z_'+feature] = (train[feature] - train[feature].mean())/train[feature].std(ddof=0)
    df['sq_'+feature] = (train[feature])**2
    df['sqrt_'+feature] = np.abs(train[feature])**(1/2)
    df['cp_'+feature] = pd.DataFrame(rankdata(train[feature]))
    df['cnp_'+feature] = pd.DataFrame((norm.cdf(train[feature])))

/home/wopr/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
for df in [df]:
#####Handling Missing Values#####     
    for i in range(len(df.columns)):
        df.iloc[:,i] = (df.iloc[:,i]).fillna(-1)

In [9]:
df.isnull().values.any()
df = reduce_mem_usage(df)

Memory usage after optimization is: 1232.91 MB
Decreased by 37.9%


In [10]:
from scipy.special import erfinv
trafo_columns = [c for c in df.columns if len(df[c].unique()) != 2]
for col in trafo_columns:
    values = sorted(set(df[col]))
    # Because erfinv(1) is inf, we shrink the range into (-0.9, 0.9)
    f = pd.Series(np.linspace(-0.9, 0.9, len(values)), index=values)
    f = np.sqrt(2) * erfinv(f)
    f -= f.mean()
    df[col] = df[col].map(f)

In [11]:
from sklearn.decomposition import PCA
pca = PCA(n_components=200)
pca.fit(df[trafo_columns])
df = pca.transform(df[trafo_columns])
df = pd.DataFrame(df)

In [12]:
df['target'] = df_original.target.values
df = reduce_mem_usage(df)
df.head(5)

Memory usage after optimization is: 153.35 MB
Decreased by 75.0%


,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,target
0,3.656250,2.972656,2.697266,1.922852,-0.316895,-2.728516,-0.391357,2.423828,-1.222656,0.296387,...,-0.744629,-1.319336,-0.296143,-0.256104,-0.559570,0.569336,-0.236450,0.621094,0.332764,0.0
1,-3.251953,-2.046875,-3.097656,-2.490234,2.380859,1.890625,-1.653320,1.649414,2.570312,1.586914,...,0.166260,0.687988,0.250488,-0.371338,-0.622559,0.479736,-0.010170,0.580078,-0.894043,0.0
2,3.796875,2.878906,-3.308594,2.134766,1.337891,1.439453,-1.758789,4.015625,-1.604492,-1.756836,...,-0.360352,0.579590,0.518066,-0.079285,-0.029160,0.211792,-0.599121,-0.657715,0.778320,0.0
3,3.775391,3.048828,2.117188,-2.343750,-0.838379,-1.784180,-2.996094,1.046875,3.380859,-0.449951,...,0.569824,0.808105,1.052734,0.145874,-0.353271,-0.773926,-0.468018,-0.250977,-0.508301,0.0
4,-2.714844,-2.474609,2.394531,1.767578,-1.456055,-1.204102,0.983887,-3.134766,1.757812,2.269531,...,-0.796387,-0.949707,-0.615234,0.566895,0.230469,-0.014931,0.400391,-0.581543,-0.197021,0.0


In [13]:
#train = df[df['target'].notnull()]
#target = train['target']
#test = df[df['target'].isnull()]
#trafo_columns = [c for c in train.columns if c not in ['target']]
#train.shape

In [14]:
trafo_columns = [c for c in df.columns if c not in ['target']]

In [16]:
from keras import backend as K
from keras.activations import elu
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from imblearn.keras import balanced_batch_generator
from imblearn.under_sampling import NearMiss, RandomUnderSampler, CondensedNearestNeighbour, AllKNN, InstanceHardnessThreshold
from sklearn.model_selection import KFold
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

learning_rate = 0.0001
mom = 0.1
dcy = 0.996
nb_folds = 4
nb_epoch = 100
batch_size = 128
encoding_dim =1500
hidden_dim = int(encoding_dim) #i.e. 7
sgd = SGD(lr=learning_rate, momentum=mom, decay=dcy)
#folds = StratifiedKFold(n_splits=nb_folds, shuffle=True, random_state=420)
folds = KFold(n_splits = nb_folds, random_state = 338, shuffle = True)
auto = np.zeros(df[trafo_columns].shape)
layer_output = np.zeros((len(df), 1500)) # change when nn shape changes
#layer_output = np.zeros(df[trafo_columns].shape)
#train_auto = np.zeros(train[trafo_columns].shape)
#test_auto = np.zeros(test[trafo_columns].shape)
predictions = np.zeros(len(df))
#label_cols = ["target"]
#y_split = train[label_cols].values

# Horovod: adjust learning rate based on number of GPUs.
opt = keras.optimizers.SGD(lr=learning_rate, decay=dcy, momentum=mom, nesterov=True)
# Horovod: add Horovod Distributed Optimizer.
opt = hvd.DistributedOptimizer(opt)

cp = ModelCheckpoint(filepath="autoencoder_0.h5",
                               save_best_only=True,
                               verbose=0)

tb = TensorBoard(log_dir='./logs',
                histogram_freq=0,
                write_graph=True,
                write_images=True)

es= EarlyStopping(monitor='val_acc',
                  min_delta=0,
                  patience=20,
                  verbose=1, mode='min')

#for fold_, (trn_idx, val_idx) in enumerate(folds.split(y_split[:,0], y_split[:,0])):
#    print("fold {}".format(fold_))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(df)):
    print("fold {}".format(fold_))

    trn_data = df[trafo_columns].iloc[trn_idx]
    val_data = df[trafo_columns].iloc[val_idx]

    def add_noise(series, noise_level):
        return series * (1 + noise_level * np.random.randn(series.shape[1]))
    
    noisy_trn_data = add_noise(trn_data, 0.04)

    input_dim = noisy_trn_data.shape[1] #num of columns, 30
    input_layer = Input(shape=(input_dim, ))
    encoder = Dense(encoding_dim, activation="tanh", activity_regularizer=regularizers.l1(learning_rate))(input_layer)
    encoder = BatchNormalization()(encoder)
    encoder = Dense(hidden_dim, activation="relu")(encoder)
    encoder = BatchNormalization()(encoder)
    decoder = Dense(hidden_dim, activation='relu')(encoder)
    decoder = Dense(input_dim, activation='tanh')(decoder)
    autoencoder = Model(inputs=input_layer, outputs=decoder)
    autoencoder.summary()
    
    autoencoder.compile(metrics=['accuracy'],
                        loss='mean_squared_error',
                        optimizer='sgd')

    cp = ModelCheckpoint(filepath="autoencoder_fraud.h5",
                                   save_best_only=True,
                                   verbose=0)

    tb = TensorBoard(log_dir='./logs',
                    histogram_freq=0,
                    write_graph=True,
                    write_images=True)
    
    es= EarlyStopping(monitor='val_acc',
                  min_delta=0,
                  patience=10,
                  verbose=1, mode='auto')

    history = autoencoder.fit(noisy_trn_data, trn_data,
                        epochs=nb_epoch,
                        batch_size=batch_size,
                        shuffle=True,
                        validation_data=(val_data, val_data),
                        verbose=1,
                        callbacks=[cp, tb, es]).history
    
    
    auto[val_idx] += autoencoder.predict(df.iloc[val_idx][trafo_columns], verbose=1)
    mse = autoencoder.predict(df[trafo_columns] / folds.n_splits, verbose=1)
    predictions += np.mean(np.power(df[trafo_columns] - mse, 2), axis=1)
    # we build a new model with the activations of the old model
    # this model is truncated after the first layer
    get_1st_layer_output = K.function([autoencoder.layers[0].input],
                                  [autoencoder.layers[1].output])
    layer_output[val_idx] += pd.DataFrame(np.concatenate(get_1st_layer_output([df.iloc[val_idx][trafo_columns]])))
    
auto_final = pd.DataFrame(auto / folds.n_splits)
hidden = pd.DataFrame(layer_output / folds.n_splits)

fold 0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1500)              301500    
_________________________________________________________________
batch_normalization_3 (Batch (None, 1500)              6000      
_________________________________________________________________
dense_6 (Dense)              (None, 1500)              2251500   
_________________________________________________________________
batch_normalization_4 (Batch (None, 1500)              6000      
_________________________________________________________________
dense_7 (Dense)              (None, 1500)              2251500   
_________________________________________________________________
dense_8 (Dense)              (None, 200)               300200    
Tot

300000/300000 [==============================] - 21s 69us/step - loss: 0.6216 - acc: 0.3281 - val_loss: 0.6281 - val_acc: 0.2852
Epoch 26/100
300000/300000 [==============================] - 20s 68us/step - loss: 0.6184 - acc: 0.3293 - val_loss: 0.6247 - val_acc: 0.3191
Epoch 00026: early stopping
400000/400000 [==============================] - 20s 49us/step
fold 2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 200)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 1500)              301500    
_________________________________________________________________
batch_normalization_7 (Batch (None, 1500)              6000      
_________________________________________________________________
dense_14 (Dense)             (None, 1500)              2251500   
_____________________________________

300000/300000 [==============================] - 21s 69us/step - loss: 0.6343 - acc: 0.3285 - val_loss: 0.6407 - val_acc: 0.3345
Epoch 23/100
300000/300000 [==============================] - 20s 67us/step - loss: 0.6304 - acc: 0.3283 - val_loss: 0.6376 - val_acc: 0.3071
Epoch 24/100
300000/300000 [==============================] - 21s 69us/step - loss: 0.6269 - acc: 0.3273 - val_loss: 0.6329 - val_acc: 0.3379
Epoch 25/100
300000/300000 [==============================] - 21s 70us/step - loss: 0.6234 - acc: 0.3274 - val_loss: 0.6269 - val_acc: 0.3263
Epoch 26/100
300000/300000 [==============================] - 21s 68us/step - loss: 0.6202 - acc: 0.3279 - val_loss: 0.6241 - val_acc: 0.3185
Epoch 27/100
300000/300000 [==============================] - 21s 69us/step - loss: 0.6171 - acc: 0.3240 - val_loss: 0.6251 - val_acc: 0.3000
Epoch 28/100
300000/300000 [==============================] - 21s 69us/step - loss: 0.6143 - acc: 0.3258 - val_loss: 0.6191 - val_acc: 0.3299
Epoch 00028: early 

In [17]:
hidden['target'] = target
hidden['ID_code'] = id.values
hidden.head(5)

,0,1,2,3,4,5,6,7,8,9,...,1492,1493,1494,1495,1496,1497,1498,1499,target,ID_code
0,-0.003360,0.001960,0.004876,-0.000782,3.782352e-03,-0.000382,-0.002048,0.001241,-0.001704,0.003050,...,0.004864,0.001233,0.000447,-0.004161,0.000504,0.006756,0.003351,0.002942,0.0,train_0
1,-0.004592,-0.002544,-0.002722,-0.000438,-1.098417e-03,0.004021,0.001530,0.001419,-0.000801,-0.002329,...,0.001542,-0.000216,0.002646,0.000715,0.002380,0.000198,0.000259,0.004444,0.0,train_1
2,-0.004127,-0.002720,-0.002631,0.001193,2.521100e-03,0.000415,0.002113,-0.000619,0.002005,-0.002453,...,0.002828,0.005525,-0.002819,-0.002474,0.001265,0.002012,0.002262,-0.000398,0.0,train_2
3,-0.001261,0.000557,-0.002861,-0.001616,5.629954e-07,0.005274,0.002290,-0.001197,0.003010,-0.000217,...,0.000507,0.001118,-0.000603,-0.000351,0.000848,0.003047,0.000592,0.000747,0.0,train_3
4,-0.003760,-0.004555,-0.000203,-0.002491,4.196662e-03,-0.002235,-0.000133,-0.000572,0.001390,-0.003354,...,-0.000753,0.001743,-0.002099,0.003958,0.000744,0.001946,0.001416,0.005175,0.0,train_4


In [18]:
%whos DataFrame

Variable         Type         Data/Info
---------------------------------------
auto_final       DataFrame                 0         1 <...>00000 rows x 200 columns]
df               DataFrame                   0         <...>00000 rows x 201 columns]
df_original      DataFrame                ID_code  targ<...>00000 rows x 202 columns]
hidden           DataFrame                   0         <...>0000 rows x 1502 columns]
noisy_trn_data   DataFrame                 0         1 <...>00000 rows x 200 columns]
test             DataFrame                ID_code      <...>00000 rows x 201 columns]
train            DataFrame                 ID_code  tar<...>00000 rows x 202 columns]
trn_data         DataFrame                 0         1 <...>00000 rows x 200 columns]
val_data         DataFrame                 0         1 <...>00000 rows x 200 columns]


In [19]:
del df, noisy_trn_data, test, train, trn_data, val_data
gc.collect()

179

In [20]:
%whos DataFrame

Variable      Type         Data/Info
------------------------------------
auto_final    DataFrame                 0         1 <...>00000 rows x 200 columns]
df_original   DataFrame                ID_code  targ<...>00000 rows x 202 columns]
hidden        DataFrame                   0         <...>0000 rows x 1502 columns]


In [21]:
#hidden
ae_columns = [c for c in hidden.columns if c not in ['ID_code', 'target']]
df_train = hidden[hidden['target'].notnull()]
target = df_train['target']
df_test = hidden[hidden['target'].isnull()]
df_train.shape

#final
#ae_columns = [c for c in auto_final.columns if c not in ['ID_code', 'target']]
#df_train = auto_final[auto_final['target'].notnull()]
#target = df_train['target']
#df_test = auto_final[auto_final['target'].isnull()]
#df_train.shape


(200000, 1502)

In [22]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import PredefinedSplit
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()
label_cols = ["target"]
y_split = df_train[label_cols].values

param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}


for fold_, (trn_idx, val_idx) in enumerate(folds.split(y_split[:,0], y_split[:,0])):
    print("fold {}".format(fold_))

    train_x, train_y = df_train[ae_columns].iloc[trn_idx], df_train['target'].iloc[trn_idx]
#    trn_data, trn_y = df_train[ae_columns].iloc[trn_idx], df_train['target'].iloc[trn_idx]
    val_x, val_y = df_train[ae_columns].iloc[val_idx], df_train['target'].iloc[val_idx]

#    classes=[]
#    for i in np.unique(trn_y):
#        classes.append(i)
#        print("Before OverSampling, counts of label " + str(i) + ": {}".format(sum(trn_y==i)))

#    sm=SMOTE(random_state=2)
#    train_x, train_y = sm.fit_sample(trn_data, trn_y.ravel())

#    print('After OverSampling, the shape of train_X: {}'.format(train_x.shape))
#    print('After OverSampling, the shape of train_y: {} \n'.format(train_y.shape))

#    for eachClass in classes:
#        print("After OverSampling, counts of label " + str(eachClass) + ": {}".format(sum(train_y==eachClass)))  
    
    num_round = 20000 
    train_data = pd.DataFrame(train_x)
    train_y = pd.DataFrame(train_y)
    train = lgb.Dataset(train_x, label=train_y)#, categorical_feature=categorical_feats)
    val = lgb.Dataset(val_x, label=val_y)#, categorical_feature=categorical_feats)
    
    # Create parameters to search
    fit_params={"early_stopping_rounds":100, 
                "eval_metric" : 'auc', 
                'eval_set' : [(val_x,val_y)],
                'eval_names': ['valid'],
                #'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
                'verbose': 100}

    clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=-1, device = 'gpu', n_estimators=5000)
    gs = RandomizedSearchCV(
        estimator=clf, param_distributions=param_test, 
        n_iter=num_round,
        scoring='roc_auc',
        cv=3,
        refit=True,
        random_state=314,
        verbose=True)
    
    gs.fit(train_data, train_y, **fit_params)
    oof[val_idx] = gs.predict(df_train.iloc[val_idx][ae_columns], num_iteration=clf.best_iteration)

    
    train_prediction = gs.predict(df_train[ae_columns] / folds.n_splits)
    predictions += gs.predict(df_test[ae_columns] / folds.n_splits)
    print("BEST PARAMETERS: " + str(gs.best_params_))
    print("BEST CV SCORE: " + str(gs.best_score_))

np.sqrt(mean_squared_error(oof, target))

fold 0
Fitting 3 folds for each of 20000 candidates, totalling 60000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/wopr/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/wopr/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training until validation scores don't improve for 100 rounds.
[100]	valid's auc: 0.754833
[200]	valid's auc: 0.781046
[300]	valid's auc: 0.792885
[400]	valid's auc: 0.800007
[500]	valid's auc: 0.804638
[600]	valid's auc: 0.807236
[700]	valid's auc: 0.809681
[800]	valid's auc: 0.81072
[900]	valid's auc: 0.81162
[1000]	valid's auc: 0.812291
[1100]	valid's auc: 0.813354
[1200]	valid's auc: 0.814101
[1300]	valid's auc: 0.814439
[1400]	valid's auc: 0.8147
[1500]	valid's auc: 0.814953
Early stopping, best iteration is:
[1425]	valid's auc: 0.815138


/home/wopr/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/wopr/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training until validation scores don't improve for 100 rounds.
[100]	valid's auc: 0.75539
[200]	valid's auc: 0.781841
[300]	valid's auc: 0.793396
[400]	valid's auc: 0.800274
[500]	valid's auc: 0.804897
[600]	valid's auc: 0.808055
[700]	valid's auc: 0.809553
[800]	valid's auc: 0.811069
[900]	valid's auc: 0.81194
[1000]	valid's auc: 0.812034
[1100]	valid's auc: 0.812615
[1200]	valid's auc: 0.812731
[1300]	valid's auc: 0.813149
[1400]	valid's auc: 0.813299
[1500]	valid's auc: 0.813459
[1600]	valid's auc: 0.813987
[1700]	valid's auc: 0.814606
[1800]	valid's auc: 0.814831
[1900]	valid's auc: 0.81518
[2000]	valid's auc: 0.815602
[2100]	valid's auc: 0.815896
[2200]	valid's auc: 0.815934
Early stopping, best iteration is:
[2125]	valid's auc: 0.816134


...

/home/wopr/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/wopr/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training until validation scores don't improve for 100 rounds.
[100]	valid's auc: 0.760343
[200]	valid's auc: 0.78224
[300]	valid's auc: 0.79291
[400]	valid's auc: 0.799567
[500]	valid's auc: 0.802868
[600]	valid's auc: 0.805677
[700]	valid's auc: 0.807678
[800]	valid's auc: 0.808776
[900]	valid's auc: 0.809588
[1000]	valid's auc: 0.810528
[1100]	valid's auc: 0.810976
[1200]	valid's auc: 0.811565
[1300]	valid's auc: 0.811618
[1400]	valid's auc: 0.812008
[1500]	valid's auc: 0.812226
[1600]	valid's auc: 0.812774
[1700]	valid's auc: 0.812958
[1800]	valid's auc: 0.813277
[1900]	valid's auc: 0.813258
Early stopping, best iteration is:
[1845]	valid's auc: 0.813474


/home/wopr/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/wopr/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training until validation scores don't improve for 100 rounds.
[100]	valid's auc: 0.757795
[200]	valid's auc: 0.781945
[300]	valid's auc: 0.793293
[400]	valid's auc: 0.799362
[500]	valid's auc: 0.803363
[600]	valid's auc: 0.805486
[700]	valid's auc: 0.80695
[800]	valid's auc: 0.808457
[900]	valid's auc: 0.809819
[1000]	valid's auc: 0.810371
[1100]	valid's auc: 0.810526
[1200]	valid's auc: 0.81075
[1300]	valid's auc: 0.811195


KeyboardInterrupt: 

In [ ]:
sub_df = pd.DataFrame({"ID_code":df_test["ID_code"].values})
sub_df["target"] = predictions
sub_df.to_csv("submission.csv", index=False)

In [ ]:
#df_original.to_csv("df_original.csv", index=False)
df_train.to_csv("auto_model_reconstructions/train_auto_0.csv", index=False)
df_test.to_csv("auto_model_reconstructions/test_auto_0.csv", index=False)